In [1]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.pipeline import merge_entities

In [2]:
nlp = spacy.load("en_core_web_sm")
# patterns = [
#     {"label":"DATE", "pattern":[{"TEXT": {"REGEX": "^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)\d{4}$"}}]}
#     {"label":"COL-DET", "pattern":}
# ]
# ruler.add_patterns(patterns)
# ruler.to_disk("../data/patterns.jsonl")

ruler = EntityRuler(nlp, overwrite_ents=True).from_disk("../data/patterns.jsonl")

In [3]:
nlp.add_pipe(ruler)
nlp.add_pipe(merge_entities)

In [4]:
inputQs = []
with open('../data/queslist.txt', 'r') as f:
    x = f.readlines()
    for t in range(0,14000,50):
        tx = x[t].strip()
        inputQs.append(tx)
    

In [5]:
# displacy.render(doc, options={'fine-grained': True})

In [6]:
batsmen = ["KL Rahul","CH Gayle","Yuvraj Singh","KK Nair","AR Patel","MP Stoinis","MA Agarwal","R Ashwin","BATSMEN","SA Yadav","E Lewis","Ishan Kishan","HH Pandya","RG Sharma","KH Pandya","AJ Finch","WP Saha","S Dhawan","KS Williamson","YK Pathan","MK Pandey","DJ Hooda","Shakib Al Hasan","Q de Kock","V Kohli","AB de Villiers","CJ Anderson","Mandeep Singh","C de Grandhomme","P Negi","Washington Sundar","UT Yadav","Mohammed Siraj","SR Watson","AT Rayudu","SK Raina","SW Billings","RA Jadeja","MS Dhoni","DJ Bravo","AD Hales","Rashid Khan","B Kumar","S Kaul","Sandeep Sharma","M Vohra","PA Patel","MM Ali","BB McCullum","CA Lynn","SP Narine","RV Uthappa","N Rana","AD Russell","KD Karthik","Shubman Gill","BCJ Cutting","JP Duminy","RK Singh","TK Curran","PP Chawla","Kuldeep Yadav","M Prasidh Krishna","F du Plessis","KV Sharma","AM Rahane","DJM Short","SV Samson","BA Stokes","JC Buttler","RA Tripathi","CR Woakes","AJ Tye","MM Sharma","Mujeeb Ur Rahman","SN Khan","Shivam Mavi","G Gambhir","JJ Roy","SS Iyer","RR Pant","GJ Maxwell","R Tewatia","V Shankar","CH Morris","Mohammed Shami","S Nadeem","TA Boult","BB Sran","M Vijay","PP Shaw","SS Iyer","HV Patel","TG Southee","KA Pollard","MJ McClenaghan","C Munro","K Gowtham","STR Binny","MK Lomror","JC Archer","JD Unadkat","AD Nath","MK Tiwary","P Chopra","R Vinay Kumar","Mohammad Nabi","Basil Thampi","M Markande","JJ Bumrah","Mustafizur Rahman","PJ Sangwan","B Stanlake","MG Johnson","DA Miller","AS Rajpoot","Harbhajan Singh","DL Chahar","M Ashwin","DR Shorey","JPR Scantlebury","H Klaasen","LE Plunkett","S Gopal","H Klaasen","DT Christian","DS Kulkarni","Abhishek Sharma","NV Ojha","A Dananjaya","SP Goswami","CR Brathwaite","IS Sodhi","Anureet Singh","B Laughlin","RK Bhui","SN Thakur","A Mishra","KM Jadhav","MA Wood","Imran Tahir","YS Chahal"]

In [7]:
x = "DL Chahar"
if str(x) in batsmen:
    print("yes")

yes


In [101]:
inputQs = ['Who was the winner between Mum Indians and Super Kings at Mumbai?',
#  'Who was the loser between Mum Indians and Super Kings at Mumbai?',
 'Who was the winner between Mum Indians and Super Kings on 07/04/2018?',
#  'Who was the loser between Mum Indians and Super Kings on 07/04/2018?',
 'When was the match between Mum Indians & Super Kings held at Mumbai?',
 'Where was the match between Mum Indians & Super Kings held on 07/04/2018?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many no balls did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many no balls did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many wide balls did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many wide balls did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many runs were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many overs did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many overs did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many no balls did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many no balls did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many wide balls did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many wide balls did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?']

In [148]:
mapEnt = {}
mapEnt["GROUND"] = "Ground"
mapEnt["DATE"] = "MatchDate"
mapEnt["PLAYER"] = "Player"
mapEnt["runs"] = "R"
mapEnt["sixes"] = "`6s`"
mapEnt["fours"] = "`4s`"
mapEnt["overs"] = "`O`"
mapEnt["winner"] = "Winner"
mapEnt["wide balls"] = "`WD`"
mapEnt["no balls"] = "`NB`"

In [170]:
def info(doc):
    displacy.render(doc, style='ent')
    print("{:13} {:10} {:20} {:20} {:20} {:20}".format("token.text", "token.pos_", "token.tag_", "token.dep_", "token.lefts", "token.rights"))
    for token in doc:
        print("{:13} {:10} {:20} {:20} {:20} {:20}".format(token.text, token.pos_, token.tag_, token.dep_, str([token.text for token in token.lefts]), str([token.text for token in token.rights])))
    print()
    print("{:25} {:25} {:25} {:25}".format("chunk.text", "chunk.root.text", "chunk.root.dep_", "chunk.root.head.text"))
    for chunk in doc.noun_chunks:
        print("{:25} {:25} {:25} {:25}".format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))
    # Finding a verb with a subject from below — good
    print()
    print("{:25} {:25}".format("ent.text", "ent.label_"))
    for ent in doc.ents:
        print("{:25} {:25}".format(ent.text, ent.label_))
    print()
    displacy.render(doc)
    return

def getType(doc):
    if doc[0].text == "How":
        return 2
    else:
        return 1

def typeProcessing(doc, qtype):
    if qtype == 1:
        pass
    if qtype == 2:
        ## Added a rule here, if query is of definite type, then merge phrase if left tree contains more than 1 ('many' is always included in the left tree)
        if(doc[3].n_lefts == 2):
            span = doc[doc[3].left_edge.i+2 : doc[3].right_edge.i+1]
            with doc.retokenize() as retokenizer:
                retokenizer.merge(span, attrs={"pos":u"NOUN", "tag":"NNS"})
    return doc

def createQuery(doc, qtype):
    batting = 0
    bowling = 0
    ## Select Query
    s_cl = "SELECT "
    root = [token for token in doc if token.dep_ == "ROOT"][0]
    if qtype == 2:
        sel = list(root.lefts)[0]
        if str(sel) == "did":
            combCl = [token for token in doc if token.dep_ == "aux"][0]
            sel = list(combCl.lefts)[0]
            s_cl += mapEnt[str(sel)]
        else:    
            s_cl += mapEnt[str(sel)]
        if str(sel) == "overs" or str(sel) == "wide balls" or str(sel) == "no balls":
            bowling = 1
    else:
        sel = list(root.rights)[0]
        s_cl += " * "
    s_cl += " FROM match_data "
   
    w_cl = "WHERE "
    team_flag = 0
    t1 = ""
    t2 = ""
  
    for ent in doc.ents:
        if w_cl[-1] != " ":
            w_cl += " AND "
        if team_flag == 0 and str(ent.label_) == "TEAM":
            t1 = ent.text
#             t_cond = "(TEAM1" + ' = "' + ent.text + '")'
            team_flag = 1
        elif team_flag == 1 and str(ent.label_) == "TEAM":
            t2 = ent.text
#             t_cond += " AND (TEAM2" + ' = "' + ent.text + '")'
#             w_cl += "("+ t_cond + ")"
            team_flag = 2
        else:
            if str(ent.label_) == "GROUND" or str(ent.label_) == "DATE":
                w_cl += "(match_data." + mapEnt[ent.label_] + " = '" + ent.text + "')"
            elif str(ent.label_) == "PLAYER" and bowling == 0:
                w_cl += "(batting_frame." + mapEnt[ent.label_] + " = '" + ent.text + "')"
                batting = 1
            else:
                w_cl += "(bowling_frame." + mapEnt[ent.label_] + " = '" + ent.text + "')"
                bowling = 1
    if team_flag == 2:
        t_cond = "(match_data.Team1 = '{}' AND match_data.Team2 = '{}')"
        x = t_cond.format(t1,t2)
        y = t_cond.format(t2,t1)
        if w_cl[-1] != " ":
            w_cl += " AND "
        w_cl += "(" + x +" OR " + y + ")"
    
    if batting == 1:
        s_cl += "INNER JOIN batting_frame on match_data.ID = batting_frame.ID "
    if bowling == 1:
        s_cl += "INNER JOIN bowling_frame on match_data.ID = bowling_frame.ID "

    
        
#     print(s_cl + w_cl)
    return s_cl + w_cl

In [171]:
import pandas as pd
import os
import re
from collections import OrderedDict
# from sqlalchemy import create_engine, MetaData, Table
import inflect
import datetime
import pandasql as ps
Matches = OrderedDict()
Batsmen = []
bowling_frame = pd.DataFrame()
batting_frame = pd.DataFrame()
# print(newDF)
Bowlers = []
Players = []
bowling_columns = ["ID", "Bowler", "O", "M", "R", "W", "Econ", "0s", "4s", "6s", "WD", "NB"]
directory = os.path.join("../data")
for root,dirs,files in os.walk(directory):
    for count, file in enumerate(files):
        if file.endswith("ing.csv"):
            df = pd.read_csv("../data/" + file)
            match_no = int(re.search(r'\d+', file).group())
            df.insert(loc=0, column="ID", value = match_no)
            if match_no not in Matches.keys():
                Matches[match_no] = OrderedDict()
            if "bowling" in file:
                
                for bowler in df['Bowling']:
                    if bowler not in Bowlers:
                        Bowlers.append(bowler)
                    if bowler not in Players:
                        Players.append(bowler)
                df = df[df.Bowling != 'Bowling']
                bowling_frame = bowling_frame.append(df, ignore_index = True)
                
            else:
                for batsman in df['BATSMEN']:
                    if batsman not in Batsmen:
                        Batsmen.append(batsman)
                    if batsman not in Players:
                        Players.append(batsman)
                df.rename(columns={'Unnamed: 1':'FOW'}, inplace=True)
                df = df[df.BATSMEN != 'BATSMEN']
                batting_frame = batting_frame.append(df, ignore_index = True)                        
bowling_frame = bowling_frame.rename(columns={'Bowling' : 'Player'})
batting_frame = batting_frame.rename(columns={'BATSMEN' : 'Player'})
# Other Match Data
match_data = pd.read_csv("../data/match-data.csv")
# Change the date format to mm/dd/yy
for index, row in match_data.iterrows():
    row['Match Date'] = datetime.datetime.strptime(row['Match Date'], '%b %d, %Y').strftime('%d/%m/%Y')
match_data.rename(columns={'Team 1':'Team1', 'Team 2': 'Team2', 'Match Date': 'MatchDate'}, inplace=True)
match_data.insert(loc=0, column='ID', value=match_data.index + 1)

In [172]:
# q1 = "SELECT winner FROM match_data WHERE (match_data.Ground = "Mumbai") AND ((match_data.Team1 = "Mum Indians" AND match_data.Team2 = "Super Kings") OR (match_data.Team1 = "Super Kings" AND match_data.Team2 = "Mum Indians"))
# q1 = sqlQuery
# q1 = "SELECT * FROM match_data WHERE (match_data.Ground = 'Mumbai') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum Indians'))"
q1 = "SELECT `4s` FROM match_data INNER JOIN batting_frame on match_data.ID = batting_frame.ID WHERE (batting_frame.Player = 'RG Sharma') AND (match_data.Ground = 'Mumbai') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum Indians'))"
print(ps.sqldf(q1))

Empty DataFrame
Columns: [4s]
Index: []


In [177]:
def Hub(questions):
    for NLquery in questions:
        doc = nlp(NLquery)
#         info(doc)
        qtype = getType(doc)
        doc = typeProcessing(doc, qtype)
#         info(doc)
        # Get relavent terms for query formation
        relv = set()
        for token in doc:
            if (token.pos_ == "PRON" or token.pos_ == "NOUN") and token.dep_ == "attr":
                relv.add(token.text)
            if token.pos_ == "ADV":
                relv.add(token.text)
            if token.pos_ == "NOUN" and (token.dep_ == "nsubjpass" or token.dep_ == "nsubj"):
                relv.add(token.text)
            if token.pos_ == "ADP" and (token.dep_ == "agent" or token.dep_ == "prep"):
                relv.add(token.text)
#         print(relv)
        sqlQuery = str(createQuery(doc, qtype))
        print(sqlQuery)
        print()
        print(ps.sqldf(sqlQuery))

In [178]:
Hub(inputQs)

SELECT  *  FROM match_data WHERE (match_data.Ground = 'Mumbai') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum Indians'))

   ID        Team1        Team2       Winner    Margin  Ground   MatchDate  \
0   1  Mum Indians  Super Kings  Super Kings  1 wicket  Mumbai  07/04/2018   

  Scorecard  
0       T20  
SELECT  *  FROM match_data WHERE (match_data.MatchDate = '07/04/2018') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum Indians'))

   ID        Team1        Team2       Winner    Margin  Ground   MatchDate  \
0   1  Mum Indians  Super Kings  Super Kings  1 wicket  Mumbai  07/04/2018   

  Scorecard  
0       T20  
SELECT  *  FROM match_data WHERE (match_data.Ground = 'Mumbai') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match

  4s
0  1
SELECT `4s` FROM match_data INNER JOIN batting_frame on match_data.ID = batting_frame.ID WHERE (batting_frame.Player = 'SR Watson') AND (match_data.MatchDate = '07/04/2018') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum Indians'))

  4s
0  1
SELECT `6s` FROM match_data INNER JOIN batting_frame on match_data.ID = batting_frame.ID WHERE (batting_frame.Player = 'SR Watson') AND (match_data.Ground = 'Mumbai') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum Indians'))

  6s
0  1
SELECT `6s` FROM match_data INNER JOIN batting_frame on match_data.ID = batting_frame.ID WHERE (batting_frame.Player = 'SR Watson') AND (match_data.MatchDate = '07/04/2018') AND ((match_data.Team1 = 'Mum Indians' AND match_data.Team2 = 'Super Kings') OR (match_data.Team1 = 'Super Kings' AND match_data.Team2 = 'Mum India